In [34]:
import pandas as pd
import numpy as np

In [38]:
from datasets import load_dataset
import pandas as pd

# Load the 'openbookqa' dataset (train split as an example)
dataset = load_dataset("allenai/openbookqa", split='train')

# Convert to a Pandas DataFrame
df = pd.DataFrame(dataset)

# Save the DataFrame to a CSV file
df.to_csv('openbookqa_train.csv', index=False)

print("CSV file has been saved as 'openbookqa_train.csv'")

CSV file has been saved as 'openbookqa_train.csv'


In [39]:
df = pd.read_csv('openbookqa_train.csv')

In [40]:
df.head()

,id,question_stem,choices,answerKey
0,7-980,The sun is responsible for,"{'text': ['puppies learning new tricks', 'chil...",D
1,7-584,When standing miles away from Mount Rushmore,"{'text': ['the mountains seem very close', 'th...",D
2,7-870,When food is reduced in the stomach,"{'text': ['the mind needs time to digest', 'ta...",C
3,7-321,Stars are,"{'text': ['warm lights that float', 'made out ...",C
4,9-732,You can make a telescope with a,"{'text': ['straw', 'Glass', 'Candle', 'mailing...",D


In [41]:
df['choices'][1]

"{'text': ['the mountains seem very close', 'the mountains are boring', 'the mountains look the same as from up close', 'the mountains seem smaller than in photographs'], 'label': ['A', 'B', 'C', 'D']}"

In [42]:
df = pd.read_csv('modified_questions.csv')

In [43]:
df.head()

,Question,Choices,Answer
0,"When standing miles away from Mount Rushmore,",The mountains seem very close\nThe mountains a...,The mountains seem smaller than in photographs


In [44]:
df['Choices'][0]

'The mountains seem very close\nThe mountains are boring\nThe mountains look the same as from up close\nThe mountains seem smaller than in photographs'

In [45]:
import pandas as pd

# Load the dataset from Hugging Face (or a CSV file if you have it locally)
from datasets import load_dataset

# Load the 'openbookqa' dataset (assuming we're modifying the train split for example)
dataset = load_dataset("allenai/openbookqa", split='train')

# Convert to a Pandas DataFrame
df = pd.DataFrame(dataset)

# Function to convert choices dictionary into the desired format
def format_choices(row):
    return "\n".join(row['choices']['text'])

# Apply the function to all rows to format the 'choices' column
df['choices'] = df.apply(format_choices, axis=1)

# Replace the answerKey column with the full text answer for each row
df['answerKey'] = df.apply(lambda row: row['choices'].split("\n")[ord(row['answerKey']) - ord('A')], axis=1)

# Rename columns for better understanding
df.rename(columns={'question_stem': 'Question', 'choices': 'Choices', 'answerKey': 'Answer'}, inplace=True)

# Save the modified dataset to a new CSV file
df.to_csv('modified_full_dataset.csv', index=False)

print("CSV file 'modified_full_dataset.csv' has been created successfully.")


CSV file 'modified_full_dataset.csv' has been created successfully.


In [46]:
df = pd.read_csv('modified_full_dataset.csv')

In [47]:
df.head()

,id,Question,Choices,Answer
0,7-980,The sun is responsible for,puppies learning new tricks\nchildren growing ...,"plants sprouting, blooming and wilting"
1,7-584,When standing miles away from Mount Rushmore,the mountains seem very close\nthe mountains a...,the mountains seem smaller than in photographs
2,7-870,When food is reduced in the stomach,the mind needs time to digest\ntake a second t...,nutrients are being deconstructed
3,7-321,Stars are,warm lights that float\nmade out of nitrate\ng...,great balls of gas burning billions of miles away
4,9-732,You can make a telescope with a,straw\nGlass\nCandle\nmailing tube,mailing tube


In [48]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the CSV file
data = pd.read_csv('modified_full_dataset.csv')

# Function to calculate the most similar question
def find_most_similar_question(input_question, data):
    # Extract the questions from the CSV file
    questions = data['Question']
    
    # Concatenate the input question to the series of existing questions
    all_questions = pd.concat([questions, pd.Series([input_question])], ignore_index=True)
    
    # Vectorize the questions and the input question using TF-IDF
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(all_questions)
    
    # Calculate cosine similarity between input question and all stored questions
    cosine_sim = cosine_similarity(vectors[-1], vectors[:-1])
    
    # Get the index of the most similar question
    most_similar_idx = cosine_sim.argmax()
    
    # Return the corresponding answer from the CSV
    return data.iloc[most_similar_idx]['Answer']

# Example usage
input_question = "telescope can be made with"
answer = find_most_similar_question(input_question, data)
print(f"Answer: {answer}")

Answer: mailing tube
